# Imports

In [1]:
## Frontend and uploads
import streamlit as st

## for Model 1: Image-to-Text / Captioning
from PIL import Image
from transformers import AutoProcessor, TFBlipForConditionalGeneration
from io import BytesIO
import IPython.display as display
import requests
import tensorflow as tf

from evaluate import load
import torch

## for Model 2: Text-to-Text
from google.cloud import aiplatform
from vertexai import preview
import vertexai
import base64
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

# Initialisation of Vertex AI

In [6]:
aiplatform.init(
    # your Google Cloud Project ID or number
    # environment default used is not set
    project='certain-cursor-419710',

    # the Vertex AI region you will use
    # defaults to us-central1
    location='europe-west3',

    # # Google Cloud Storage bucket in same region as location
    # # used to stage artifacts
    # staging_bucket='gs://my_staging_bucket',

    # # custom google.auth.credentials.Credentials
    # # environment default credentials used if not set
    # credentials=my_credentials,

    # # customer managed encryption key resource name
    # # will be applied to all Vertex AI resources if set
    # encryption_spec_key_name=my_encryption_key_name,

    # # the name of the experiment to use to track
    # # logged metrics and parameters
    # experiment='Storybook_Gemini',

    # # description of the experiment above
    # experiment_description='Testing Storybook project with using Vertex AI for employ of Gemini 1.5'
)

# Frontend Streamlit

In [ ]:
st.set_page_config(
    page_title="Image Captioning App",
    page_icon="🖼️",
    layout="centered",
    initial_sidebar_state="auto"

# Load Model and Preprocessor

In [ ]:
if "model" not in st.session_state:
    st.session_state.model = TFBlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base") # initialize the session state variable
if "processor" not in st.session_state:
    st.session_state.processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base") # initialize the session state variable


# Upload images and create captions

In [ ]:
# App title
st.title("Image Captioning App")

# Image upload feature
uploaded_files = st.file_uploader("Upload up to 5 images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

if uploaded_files:
    if len(uploaded_files) > 5:
        st.warning("You can only upload up to 5 images.")
    else:
        captions = []
        for uploaded_file in uploaded_files:
            # Open the image
            image = Image.open(uploaded_file)

            # Display the image
            # st.image(image, caption='Uploaded Image', use_column_width=True)

            # Preprocess the image
            # text = "A picture of"
            # inputs = processor(images=image, text=text, return_tensors="tf")
            inputs = st.session_state.processor(images=image, return_tensors="tf")

            # Generate caption
            # with tf.device('/CPU:0'):  # Use CPU to avoid potential issues with GPU
            outputs = st.session_state.model.generate(**inputs)

            # Decode and display the caption
            caption = st.session_state.processor.decode(outputs[0], skip_special_tokens=True)
            captions.append(caption)
        st.write(f"**Captions:** {captions}")


# Test Gemini Output

In [7]:
def generate():
  vertexai.init(project="certain-cursor-419710", location="europe-west3")
  model = GenerativeModel(
    "gemini-1.5-flash-001",
  )
  responses = model.generate_content(
      ["""Write a harmless children story with less than 250 words. Include a white rabbit in it."""],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    print(response.text, end="")


generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generate()

Lily was skipping through the meadow, her pink dress swirling around her. Suddenly, a flash of white caught her eye. A fluffy white rabbit with pink nose hopped out from behind a bush.

"Hello!" Lily called out softly.

The rabbit twitched its nose and hopped closer. Lily held out her hand, and the rabbit sniffed it curiously.

"You're so soft," Lily whispered, stroking its fur gently. The rabbit closed its eyes and nuzzled against her hand.

Lily giggled and pulled out a juicy carrot from her basket. "Would you like some of this?"

The rabbit took the carrot and munched happily, looking up at Lily with grateful eyes. 

Lily spent the rest of the afternoon playing with her new furry friend, feeling happy and content. As the sun began to set, the rabbit hopped away, leaving Lily with a smile on her face and a warm feeling in her heart. 
